In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cmdstanpy as stan
import os
np.set_printoptions(legacy="1.25")

df = pd.read_csv('siaya_CFP.csv')
df = df.assign(mort72 = df['hr72'] / df['total'])
df.head(3)

#downcasting happening automatically - will be deprecated in future python
repl = df['generation'].replace(         #replace generation with integer for STAN
    to_replace=df['generation'].unique(),
    value=range(1, 5)
    )


/tmp/ipykernel_160486/4177433217.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  repl = df['generation'].replace(         #replace generation with integer for STAN


In [47]:
mort_ind = []
for row in range(len(df)):
    mort_ind.extend(np.repeat(1, df['hr72'][row]))
    mort_ind.extend(np.repeat(0 , df['total'][row] - df['hr72'][row]))

replicate = []
for row in range(len(df)):
    replicate.extend(np.repeat(repl[row], df['total'][row]))

dose = []
for row in range(len(df)):
    dose.extend(np.repeat(df['Dose'][row], df['total'][row]))

logisticdata =  {
    "D": 1,
    "N": len(mort_ind),
    "L": 4,
    "y": mort_ind,
    "ll": replicate,
    "x": dose,
}
stan.write_stan_json("data.json", logisticdata)

In [15]:
#building the dictionary of data for STAN
bottle_data = {
    "D": 1,
    "N": len(mort_ind),
    "L": 4, #figure out how to get this automatically from unique values of 'generation'
    "y": mort_ind,
    "ll": replicate,
    "x": dose
}
stan.write_stan_json("data.json", bottle_data)

In [48]:
my_stanfile = os.path.join('.', 'logistic.stan')
my_model = stan.CmdStanModel(stan_file=my_stanfile, cpp_options={'STAN_THREADS':'true'})
my_model.name
my_model.stan_file
my_model.exe_file
my_model.code()
my_model.sample(data='data.json', show_console=True, iter_sampling=2000, 
                chains=1, output_dir=os.path.join('.', 'runfiles'), inits=0)

# bern_stanfile = os.path.join('.', 'bernoulli_simple.stan')
# bern_model = stan.CmdStanModel(stan_file=bern_stanfile, cpp_options={'STAN_THREADS':'true'})

# bern_model.sample(data='data.json', show_console=True, iter_sampling=2000, 
#                 chains=1, output_dir=os.path.join('.', 'runfiles'), inits=0)


09:01:44 - cmdstanpy - INFO - Chain [1] start processing
09:01:45 - cmdstanpy - INFO - Chain [1] done processing
09:01:45 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 11 Unknown error -11


Chain [1] 


RuntimeError: Error during sampling:

Command and output files:
RunSet: chains=1, chain_ids=[1], num_processes=1
 cmd (chain 1):
	['/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/logistic', 'id=1', 'random', 'seed=88619', 'data', 'file=data.json', 'init=0', 'output', 'file=/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/runfiles/logistic-20250501090144.csv', 'method=sample', 'num_samples=2000', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[-11]
 per-chain output files (showing chain 1 only):
 csv_file:
	/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/runfiles/logistic-20250501090144.csv
 console_msgs (if any):
	/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/runfiles/logistic-20250501090144_0-stdout.txt